In [1]:
pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.11.tar.gz (3.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-win_amd64.whl size=1073112 sha256=f5ba1a9131069377bcaa212dba87f0426c68ca836fb5d5adeae877ae614fb9fd
  Stored in directory: c:\users\iamthewizard\appdata\local\pip\cache\wheels\dc\42\77\a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install unstructured

     ---------------------------------------- 1.7/1.7 MB 4.4 MB/s eta 0:00:00
     ------------------------------------- 981.5/981.5 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached lxml-4.9.3-cp310-cp310-win_amd64.whl (3.8 MB)
     ---------------------------------------- 199.4/199.4 kB ? eta 0:00:00
     ---------------------------------------- 358.9/358.9 kB ? eta 0:00:00
     ---------------------------------------- 275.1/275.1 kB ? eta 0:00:00
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
     ---------------------------------------- 1.8/1.8 MB 4.8 MB/s eta 0:00:00
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using legacy 'setup.py install' for langdetect, since package 'wheel' is not installed.
  Running setup.py install for langdetect: started
  Running setup.py install for langdetect: finished with st


[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.document_loaders import JSONLoader

from IPython.display import display, HTML
import json
import time
from pathlib import Path

*<b>Create a Database of information gathered from local text files<b>*

In [2]:
# Define what Documents to load

text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader('./', glob = 'COI_txt_trial.txt', loader_cls = TextLoader, loader_kwargs=text_loader_kwargs)

'''
file_path='./150_lawergpt_dataset_qna_v1_train.jsonl'
documents = json.loads(Path(file_path).read_text())
'''
'''
loader = JSONLoader(
    file_path='./150_lawergpt_dataset_qna_v1_train.jsonl',
    jq_schema='.content',
    text_content=False,
    json_lines=True)
'''

'''
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader

folder_path = "./150_lawergpt_dataset_qna_v1_train.jsonl"
directory_loader = DirectoryLoader(folder_path)
documents = directory_loader.load()
'''
# Interpret information in the Documents

documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
texts = splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2', model_kwargs = {'device':'cpu'})

# Cretae and save the local Database

db = FAISS.from_documents(texts, embeddings)
db.save_local('faiss')

In [3]:
doc_sources = [doc.metadata['source']  for doc in documents]
doc_sources

['COI_txt_trial.txt']

*<b>Read the database of information from local text files and uses a large language model to answer questions about their content.<b>*

In [4]:
# Preparing the template

template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
"""

# Load the Language Model

local_llm_model_name = 'llama-2-13b-chat.Q5_K_M.gguf'
#llm = CTransformers(model = './llama-2-13b-chat.Q5_K_M.gguf', model_type = 'llama', config = {'max_new_tokens':256, 'temperature':0.01})
llm = LlamaCpp(
    model_path = f"./{local_llm_model_name}",
    n_gpu_layers = 20,
    n_batch = 512,
    n_ctx = 2048,
    f16_kv = True,
    verbose = True,
)


# Load the interpreted information from local Database

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
db = FAISS.load_local("faiss", embeddings)


# Prepeare a version of the llm pre-loaded with the local content

retriever = db.as_retriever(search_kwargs = {'k': 2})
prompt = PromptTemplate(template = template, input_variables = ['context', 'question'])
qa_llm = RetrievalQA.from_chain_type(llm = llm, chain_type = 'stuff', retriever = retriever, return_source_documents = True, chain_type_kwargs = {'prompt': prompt})


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


*<b>Query<b>*

In [5]:
def query(model, question):
    time_start = time.time()
    question = 'Answer like a legal consultant to this query and expand upon the laws involved - ' + question
    output = model({'query': question})
    response = output["result"]
    time_elapsed = time.time() - time_start
    display(HTML(f'<code>{local_llm_model_name} response time: {time_elapsed:.02f} sec</code>'))
    display(HTML(f'<strong>Question:</strong> {question}'))
    display(HTML(f'<strong>Answer:</strong> {response}'))

In [9]:
query(qa_llm, 'what punishment will I face if I commit arson, and how long is the jail time and fine in indian rupees?')

Llama.generate: prefix-match hit


In [10]:
query(qa_llm, 'what punishment will I face if I commit tax fraud, and how long is the jail time and fine in indian rupees?')

Llama.generate: prefix-match hit


# Retrieval Augmented Knowledge Fusion

In [ ]:
system_prompt = """
You are an assistant that helps to generate text to form nice and human understandable answers based.
The latest prompt contains the information, and you need to generate a human readable response based on the given information.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
Do not add any additional information that is not explicitly provided in the latest prompt.
I repeat, do not add any information that is not explicitly given.
"""

In [ ]:
def generate_user_prompt(question, context):
   return f"""
   The question is {question}
   Answer the question by using the provided information:
   {context}
   """

In [ ]:
def retrieve_context(question, k=3):
  data = run_query("""
    // retrieve the embedding of the question
    CALL apoc.ml.openai.embedding([$question], $apiKey) YIELD embedding
    // match relevant movies
    MATCH (m:Movie)
    WITH m, gds.similarity.cosine(embedding, m.embedding) AS score
    ORDER BY score DESC
    // limit the number of relevant documents
    LIMIT toInteger($k)
    // retrieve graph context
    MATCH (m)--()--(m1:Movie)
    WITH m,m1, count(*) AS count
    ORDER BY count DESC
    WITH m, apoc.text.join(collect(m1.title)[..3], ", ") AS similarMovies
    MATCH (m)-[r:ACTED_IN|DIRECTED]-(t)
    WITH m, similarMovies, type(r) as type, collect(t.name) as names
    WITH m, similarMovies, type+": "+reduce(s="", n IN names | s + n + ", ") as types
    WITH m, similarMovies, collect(types) as contexts
    WITH m, "Movie title: "+ m.title + " year: "+coalesce(m.released,"") +" plot: "+ coalesce(m.tagline,"")+"\n" +
          reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") + "similar movies:" + similarMovies + "\n" as context
    RETURN context
  """, {'question': question, 'k': k, 'apiKey': openai_api_key})
  return data['context'].to_list()

In [ ]:
def generate_answer(question):
    # Retrieve context
    context = retrieve_context(question)
    # Print context
    for c in context:
        print(c)
    # Generate answer
    response = run_query(
        """
  CALL apoc.ml.openai.chat([{role:'system', content: $system},
                      {role: 'user', content: $user}], $apiKey) YIELD value
  RETURN value.choices[0].message.content AS answer
  """,
        {
            "system": system_prompt,
            "user": generate_user_prompt(question, context),
            "apiKey": openai_api_key,
        },
    )
    return response["answer"][0]